In [ ]:
import sys
#Fix problem with ros python path
sys.path.remove('/opt/ros/kinetic/lib/python2.7/site-packages')

In [ ]:
import re
import os, sys
%load_ext autoreload
from CNNRobotLocalisation.Utils.utils import *

## Average training time

In [ ]:
with open('/media/data/Dokumente/Nextcloud/Studium/Bachelorarbeit/DataPreprocessing/tf_training_output.txt') as f:
    lines = f.read().splitlines()
#print(lines)
times = []
for l in lines:
    times.append(float(re.search('\((.*) sec/step', l).group(1)))
print("Average runtime: " + str(sum(times) / float(len(times))))

## Export video frames

In [ ]:
background_videos = get_recursive_file_list('/media/data/LocalizationDataNew/Videos/180417/height', 
                                            file_extensions=".avi")
for f in background_videos:
    out = '/media/data/LocalizationDataNew/Output/VideoFrames/'+f.rsplit('/',1)[1].replace('.avi','/')
    save_video_frames(f, out, mod_frames=5)

## Color transformation

In [ ]:
from PIL import ImageEnhance, Image
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

image = Image.open('/media/data/LocalizationData/RobotCrops/ManualSelection180413/Copter/180413_copter_red_4_rot_obj0/green_blue/frame000643-object00.png').convert('RGBA')

# Brightness: 0.8 - 1.6
# Contrast: 1.0 - 1.6

for i in range(10):
    for j in range(5,6):
        brightness = i / 5.0
        contrast = j / 5.0
        enhancer = ImageEnhance.Brightness(image)
        new_img = enhancer.enhance(brightness)
        enhancer = ImageEnhance.Contrast(new_img)
        new_img = enhancer.enhance(contrast)
        print("Brightness: {0}; Contrast: {1}".format(brightness,contrast))
        plt.imshow(new_img)
        plt.show()

## Class dir to csv

In [ ]:
from object_detection.utils import label_map_util
PATH_TO_LABELS = "/media/data/Dokumente/Nextcloud/Studium/Bachelorarbeit/CNNRobotLocalisation/LabelMaps/robot_label_map.pbtxt"
labelMapDict = label_map_util.get_label_map_dict(PATH_TO_LABELS)

path = "/media/data/LocalizationData/Output/ValidationSecondStage/Crops180413Backup"
files = get_recursive_file_list(path, file_extensions=['.jpg'])
csv_rows = []
for f in files:
    name = f.rsplit('/', 1)[-1]
    cat_text = 'copter_'+f.rsplit('/',2)[1]
    cat = labelMapDict[cat_text]
    csv_rows.append((name,cat_text,cat,0.0))
writeCSV(path+"/0groundtruth.csv",csv_rows)

### Number of records in tfrecord

In [ ]:
import tensorflow as tf
c = 0
for fn in ['/media/data/Dokumente/Nextcloud/Freigabe/DeepVision/realreal_objects.tfrecord']:
    for record in tf.python_io.tf_record_iterator(fn):
        c += 1
    print(fn + ': ' + str(c))